In [ ]:
import RNA

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def makeHelix(max_len,
                avg_helix_len = 3,
                avg_loop_len = 2,
                paired_emissions = [.80,.18,.02],
                ):

    
    rng = np.random.uniform()
    
    loop_len = np.random.poisson(lam=avg_loop_len)+2
    max_len -= loop_len
    
    if max_len < 6:
        return ''
    
    helix_len = np.random.poisson(lam=avg_helix_len)+3
    helix_counter = 0
    
    left_seq = ''
    right_seq = ''
            
    while (helix_counter < helix_len) & (len(left_seq) + len(right_seq) < max_len - 1):

        if np.random.uniform() < paired_emissions[0]:
            left_seq += '('
            right_seq = ')' + right_seq
        elif rng < sum(paired_emissions[:2]):
            
            max_new_seq = max_len - len(left_seq) - len(right_seq)
            
            if max_new_seq > 8:
                if np.random.uniform() < 0.5:
                    left_seq += makeHelix(max_new_seq)
                else:
                    right_seq = makeHelix(max_new_seq) + right_seq
                    
        elif np.random.uniform() < 0.5:
            left_seq += '.'
        else:
            right_seq = '.' + right_seq

        helix_counter += 1
    
    return left_seq + '.'*loop_len + right_seq

In [ ]:
def CFRNASSM(seq_len, 
             state_space = {0:'paired',1:'unpaired'},
             avg_helix_len = 3,
             avg_loop_len = 3,
             state_transitions = np.array([[.5,.5],[.3,.7]]),
             paired_emissions = [.95,.05]):
    
    current_state = int(np.random.uniform() > 0.5)
    
    left_seq = ''
    right_seq = ''
    
    while len(left_seq) + len(right_seq) < seq_len - 1:
        
        rng = np.random.uniform()
        
        helix_len = np.random.poisson(lam=avg_helix_len)+3
        loop_len = np.random.poisson(lam=avg_loop_len)
        
        if current_state == 0:
            if rng < 0.5:
                left_seq += makeHelix(seq_len - len(left_seq) - len(right_seq))
            else:
                right_seq = makeHelix(seq_len - len(left_seq) - len(right_seq)) + right_seq
        else:
            loop_counter = 0
            
            while (loop_counter < loop_len)\
                    & (len(left_seq) + len(right_seq) < seq_len - 1):
                
                if rng < 0.5:
                    left_seq += '.'
                else:
                    right_seq = '.' + right_seq
                    
                loop_counter += 1
        
        current_state = int(state_transitions[current_state,0] < rng)
    
    seq = left_seq + right_seq
    
    if len(seq) < seq_len:
        seq += '.'
    
    return seq

In [ ]:
def parseDotBracket(dot_bracket,
                    bp_dict = {'A':'U','U':'A','G':'C','C':'G'},
                    GU_prob = 0.1,
                    GU_dict = {'A':'U','U':'G','G':'U','C':'G'}
                   ):
    seq = [None]*len(dot_bracket)
    
    helix_stack = []
    
    max_iter = len(dot_bracket)
    counter = 0
    
    while (len(seq) > 0) & (counter < max_iter):
        nuc = np.random.choice(['A','U','G','C'])
        
        if dot_bracket[0] == '.':
            seq[counter] = nuc
            dot_bracket = dot_bracket[1:]
            
        elif dot_bracket[0] == '(':
            helix_stack += [(counter,np.random.choice(['A','U','G','C','G','C']))]
            dot_bracket = dot_bracket[1:]
            
        elif dot_bracket[0] == ')':
            seq[helix_stack[-1][0]] = helix_stack[-1][1]    
            
            if np.random.uniform() < GU_prob:
                seq[counter] = GU_dict[helix_stack[-1][1]]
            else:
                seq[counter] = bp_dict[helix_stack[-1][1]]
                
            helix_stack = helix_stack[:-1]      
            dot_bracket = dot_bracket[1:]
        
#         print(dot_bracket,[f[0] for f in helix_stack])
        counter += 1
    
    return ''.join(seq)

In [ ]:
struct = CFRNASSM(150)
struct

In [ ]:
# %%timeit
# RNA.inverse_fold(parseDotBracket(struct),struct)

In [ ]:
# inv_fold

In [ ]:
# %timeit RNA.fold_compound(parseDotBracket(struct)).pf()

In [ ]:
struct = CFRNASSM(150)
struct

In [ ]:
fc = RNA.fold_compound(parseDotBracket(struct))
fc.pf()
np.round(np.sum(np.array(fc.bpp()) + np.array(fc.bpp()).T,axis=1)[1:],3)

In [ ]:
plt.stem(np.sum(np.array(fc.bpp()) + np.array(fc.bpp()).T,axis=1)[1:])

plt.show()

In [5]:
from joblib import Parallel, delayed

In [6]:
def makeSamplesAndWrite(i,max_len = 1000):
    
    output = ''
    
    seq_lens = np.random.randint(150,400,size=max_len)
    
    for seq_len in seq_lens:
        
        struct = CFRNASSM(seq_len)
        seq = parseDotBracket(struct)
        fc = RNA.fold_compound(seq)
        fc.pf()
        bpp = np.sum(np.array(fc.bpp()) + np.array(fc.bpp()).T,axis=1)[1:]
        
        output += f"{seq}\t{','.join(bpp.astype('str'))}\n"
        
    with open(f'sequence_files/{i}.tsv','w') as f:
        f.write(output)

In [7]:
Parallel(n_jobs=-2,verbose=10)(delayed(makeSamplesAndWrite)(i) for i in range(100))

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done   2 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-2)]: Done   7 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-2)]: Done  12 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-2)]: Done  19 tasks      | elapsed: 27.7min
[Parallel(n_jobs=-2)]: Done  26 tasks      | elapsed: 35.7min
[Parallel(n_jobs=-2)]: Done  35 tasks      | elapsed: 47.5min
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed: 59.4min
[Parallel(n_jobs=-2)]: Done  55 tasks      | elapsed: 75.2min
[Parallel(n_jobs=-2)]: Done  66 tasks      | elapsed: 88.1min
[Parallel(n_jobs=-2)]: Done  79 tasks      | elapsed: 106.5min
[Parallel(n_jobs=-2)]: Done  92 tasks      | elapsed: 122.8min
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed: 133.1min finished


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]